## U-Net training for DRIVE challenge

Training and evaluation over the [Drive dataset](https://drive.grand-challenge.org/) of the U-Net model created with PyEDDL library.

**Francisco Javier Blázquez Martínez** \
**francisco.blazquezmartinez@epfl.ch**

Double degree in Mathematics-Computer Engineering

Complutense University of Madrid, Spain \
École Politechnique Fédérale de Lausanne, Switzerland

DeepHealth project

In [ ]:
import pyeddl.eddl as eddl
from pyeddl.tensor import Tensor

In [ ]:
# Set the following variable to the folder containing the DRIVE dataset
DRIVE_PATH = None

#### DRIVE Dataset download

In [ ]:
# Uncomment to download the DRIVE dataset
# eddl.download_drive()

#### DRIVE Data Augmentation

In [ ]:
# TODO

#### DRIVE Data Preprocessing

In [ ]:
# WARNING! Run only once, new tensor files (.bin) are created in the dataset folder

# We follow here the preprocessing done in pyeddl examples, consider however resizing 
# the images to a lower quality so that the memory required for training is much lower.

In [ ]:
import os

In [ ]:
# Get the image and mask and scale and crop (to 512x512) them both
in_1  = eddl.Input([3, 584, 584])
in_2  = eddl.Input([1, 584, 584])
layer = eddl.Concat([in_1, in_2])

layer = eddl.RandomCropScale(layer, [0.9, 1.0])
layer = eddl.CenteredCrop(layer, [512, 512])
img   = eddl.Select(layer, ["0:3"])
mask  = eddl.Select(layer, ["3"])

danet = eddl.Model([in_1, in_2], [])
eddl.build(danet,cs=eddl.CS_CPU())
eddl.summary(danet)

In [ ]:
drive_imgs = Tensor.load(DRIVE_PATH+"drive_trX.bin")
drive_msks = Tensor.load(DRIVE_PATH+"drive_trY.bin")

In [ ]:
eddl.forward(danet, [drive_imgs, drive_msks])
drive_imgs_processed = eddl.getOutput(img)
drive_msks_processed = eddl.getOutput(mask)

In [ ]:
# Resize check
#print(drive_imgs_processed.shape)
#print(drive_msks_processed.shape)

In [ ]:
drive_imgs_processed.save("drive_trX_preprocessed.bin")
drive_msks_processed.save("drive_trY_preprocessed.bin")

In [ ]:
os.rename("drive_trX_preprocessed.bin", DRIVE_PATH+"drive_trX_preprocessed.bin")
os.rename("drive_trY_preprocessed.bin", DRIVE_PATH+"drive_trY_preprocessed.bin")

#### DRIVE Preprocessed data visualization

In [ ]:
import numpy   as np
import matplotlib.pyplot as plt
import random

In [ ]:
tensor_x = Tensor.load(DRIVE_PATH + "drive_trX_preprocessed.bin")
tensor_y = Tensor.load(DRIVE_PATH + "drive_trY_preprocessed.bin")

In [ ]:
fig = plt.figure(figsize=(16, 10))
rows = 2
columns = 4

for i in range(columns):
    idx = random.randint(0, tensor_x.shape[1])
    
    img_np = tensor_x.select([str(i)])
    msk_np = tensor_y.select([str(i)])
    
    # Remove batches axis (and number of channels for the mask)
    img_np.reshape_(img_np.shape[1:])
    msk_np.reshape_(msk_np.shape[2:])
    
    # Prepare to Matplotlib visualization
    img_np = img_np.getdata().astype(np.int32).T
    msk_np = msk_np.getdata().astype(np.int32).T

    # Size check
    #print(img0_np.shape)
    #print(msk0_np.shape)

    fig.add_subplot(rows, columns, i+1)
    plt.imshow(img_np)
    plt.axis('off')
    
    fig.add_subplot(rows, columns, i+1+columns)
    plt.imshow(msk_np) # set cmap="gray" to display the image in B&W
    plt.axis('off')

#### U-Net model train

In [ ]:
# WARNING! Heavy computation, run preferably on the server!

In [ ]:
from unet import unet

In [ ]:
EPOCHS        = 10
BATCH_SIZE    = 8
NUM_BATCHES   = 50
GPU           = True
MEM           = "full_mem"
LOSS_FUNCTION = "mse"
METRICS       = "mse"
LEARNING_RATE = 0.00001

In [ ]:
in_ = eddl.Input([3, 512, 512])
net = unet(in_)

# TODO: Should we return the model created or the output layer?
#out = unet(in_) 
#net = eddl.Model([in_], [out])

eddl.build(
    net,
    eddl.adam(LEARNING_RATE),  # Optimizer
    [LOSS_FUNCTION],           # Losses
    [METRICS],                 # Metrics
    eddl.CS_GPU(mem=MEM) if GPU else eddl.CS_CPU(mem=MEM)
)

eddl.summary(net)

In [ ]:
x_train = Tensor.load(DRIVE_PATH+"drive_trX_preprocessed.bin")
x_train.div_(255.0)
x_train.info()

y_train = Tensor.load(DRIVE_PATH+"drive_trY_preprocessed.bin")
y_train.div_(255.0)
y_train.info()

xbatch = Tensor([BATCH_SIZE, 3, 512, 512])
ybatch = Tensor([BATCH_SIZE, 1, 512, 512])

In [ ]:
for i in range(EPOCHS):
    print("\nEpoch %d/%d" % (i + 1, EPOCHS))
    eddl.reset_loss(net)
    for j in range(NUM_BATCHES):
        eddl.next_batch([x_train, y_train], [xbatch, ybatch])
        eddl.train_batch(net, [xbatch_da], [ybatch_da])
        eddl.print_loss(net, j)

#### U-Net model save

In [ ]:
eddl.save("models/unet_drive.bin")

#### U-Net model load

In [ ]:
from unet import unet

In [ ]:
GPU           = True
MEM           = "full_mem"
LOSS_FUNCTION = "mse"
METRICS       = "mse"
LEARNING_RATE = 0.00001

In [ ]:
in_ = eddl.Input([3, 512, 512])
net = unet(in_)

# TODO: Should we return the model created or the output layer?
#out = unet(in_) 
#net = eddl.Model([in_], [out])

eddl.build(
    net,
    eddl.adam(LEARNING_RATE),  # Optimizer
    [LOSS_FUNCTION],           # Losses
    [METRICS],                 # Metrics
    eddl.CS_GPU(mem=MEM) if GPU else eddl.CS_CPU(mem=MEM)
)

eddl.summary(net)

In [ ]:
eddl.load(net, "models/unet_drive.bin")

#### U-Net model visual analysis

In [ ]:
import numpy   as np
import matplotlib.pyplot as plt
import random

In [ ]:
tensor_x = Tensor.load(DRIVE_PATH + "drive_trX_preprocessed.bin")
tensor_y = Tensor.load(DRIVE_PATH + "drive_trY_preprocessed.bin")

In [ ]:
fig = plt.figure(figsize=(24, 10))
rows = 3
columns = 6

for i in range(columns):
    idx = random.randint(0, tensor_x.shape[1])
    
    img_np = tensor_x.select([str(i)])
    msk_np = tensor_y.select([str(i)])
    
    # Remove batches axis (and number of channels for the mask)
    img_np.reshape_(img_np.shape[1:])
    msk_np.reshape_(msk_np.shape[2:])
    
    # Prepare to Matplotlib visualization
    img_np = img_np.getdata().astype(np.int32).T
    msk_np = msk_np.getdata().astype(np.int32).T

    # Size check
    #print(img0_np.shape)
    #print(msk0_np.shape)

    fig.add_subplot(rows, columns, i+1)
    plt.imshow(img_np)
    plt.axis('off')
    
    fig.add_subplot(rows, columns, i+1+columns)
    plt.imshow(msk_np) # set cmap="gray" to display the image in B&W
    plt.axis('off')
    
    # TODO: Here we should take the mask that our model gives!
    fig.add_subplot(rows, columns, i+1+columns*2)
    plt.imshow(msk_np) # set cmap="gray" to display the image in B&W
    plt.axis('off')

#### U-Net model evaluation

In [ ]:
# TODO: Keep some images out of the train set for evaluation!

#### References:
https://drive.grand-challenge.org/ \
https://paperswithcode.com/dataset/drive 